In [1]:
import awkward as ak
from coffea.nanoevents import NanoEventsFactory, NanoAODSchema
import awkward as ak
import matplotlib.pyplot as plt
import numpy as np

In [87]:
fname = "root://eos.cms.rcac.purdue.edu//store/mc/RunIISummer20UL18NanoAODv9/GluGluHToMuMu_M125_TuneCP5_13TeV-powheg-pythia8/NANOAODSIM/106X_upgrade2018_realistic_v16_L1v1-v1/2810000/C4DAB63C-E2A1-A541-93A8-3F46315E362C.root"
# fname = "root://eos.cms.rcac.purdue.edu//store/mc/RunIISummer20UL18NanoAODv9/DYJetsToLL_M-50_TuneCP5_13TeV-amcatnloFXFX-pythia8/NANOAODSIM/106X_upgrade2018_realistic_v16_L1v1-v2/40000/ECB51118-0153-2F40-BB6D-0204F0EE98C2.root"

events = NanoEventsFactory.from_root(
    {fname: "Events"},
    schemaclass=NanoAODSchema,
    metadata={"dataset": "DYJets"},
).events()
is_equal = events.Muon.fsrPhotonIdxG == events.Muon.fsrPhotonIdx
print(f"number of idxs equal: {ak.sum(is_equal).compute()}")
print(f"number of idxs not equal: {ak.sum(~is_equal).compute()}")

number of idxs equal: 1816181
number of idxs not equal: 48158


In [86]:
# events.FsrPhoton?
# events.Electron.matched_fsrPhoton
# events.Muon.matched_fsrPhoton?
events.Muon.fsrPhotonIdx?

Type:            Array
String form:     dask.awkward<fsrPhotonIdx, npartitions=1>
File:            /depot/cms/kernels/coffea2024/lib/python3.10/site-packages/dask_awkward/lib/core.py
Docstring:       Index of the associated FSR photon
Class docstring:
Partitioned, lazy, and parallel Awkward Array Dask collection.

The class constructor is not intended for users. Instead use
factory functions like :py:func:`~dask_awkward.from_parquet`,
:py:func:`~dask_awkward.from_json`, etc.

Within dask-awkward the ``new_array_object`` factory function is
used for creating new instances.

In [83]:
events.Muon.fields

['dxy',
 'dxyErr',
 'dxybs',
 'dz',
 'dzErr',
 'eta',
 'ip3d',
 'jetPtRelv2',
 'jetRelIso',
 'mass',
 'miniPFRelIso_all',
 'miniPFRelIso_chg',
 'pfRelIso03_all',
 'pfRelIso03_chg',
 'pfRelIso04_all',
 'phi',
 'pt',
 'ptErr',
 'segmentComp',
 'sip3d',
 'softMva',
 'tkRelIso',
 'tunepRelPt',
 'mvaLowPt',
 'mvaTTH',
 'charge',
 'jetIdx',
 'nStations',
 'nTrackerLayers',
 'pdgId',
 'tightCharge',
 'fsrPhotonIdx',
 'highPtId',
 'highPurity',
 'inTimeMuon',
 'isGlobal',
 'isPFcand',
 'isStandalone',
 'isTracker',
 'jetNDauCharged',
 'looseId',
 'mediumId',
 'mediumPromptId',
 'miniIsoId',
 'multiIsoId',
 'mvaId',
 'mvaLowPtId',
 'pfIsoId',
 'puppiIsoId',
 'softId',
 'softMvaId',
 'tightId',
 'tkIsoId',
 'triggerIdLoose',
 'genPartIdx',
 'genPartFlav',
 'cleanmask',
 'fsrPhotonIdxG',
 'genPartIdxG',
 'jetIdxG']

In [19]:
# events.Electron.matched_photon?
events.Electron.photonIdxG?

Type:            Array
String form:     dask.awkward<photonIdxG, npartitions=1>
File:            /depot/cms/kernels/coffea2024/lib/python3.10/site-packages/dask_awkward/lib/core.py
Docstring:       index of the associated photon (-1 if none)
Class docstring:
Partitioned, lazy, and parallel Awkward Array Dask collection.

The class constructor is not intended for users. Instead use
factory functions like :py:func:`~dask_awkward.from_parquet`,
:py:func:`~dask_awkward.from_json`, etc.

Within dask-awkward the ``new_array_object`` factory function is
used for creating new instances.

In [20]:
print(events.Muon.fsrPhotonIdxG.compute())
print(events.Electron.photonIdxG.compute())

[[0], [-1, -1], [-1, -1], [-1], [-1], ..., [-1], [-1, -1], [-1, -1, -1], [-1]]
[[], [], [], [], [-1], [], [-1], [], [], ..., [], [-1], [], [], [], [], [], []]


In [63]:
mu_photon_idxs = events.Muon.fsrPhotonIdxG[events.Muon.fsrPhotonIdxG != -1][:,:1]
mu_photon_idxs = ak.flatten(ak.pad_none(mu_photon_idxs,1))
el_photon_idxs = events.Electron.photonIdxG[events.Electron.photonIdxG != -1][:,:1]
el_photon_idxs = ak.flatten(ak.pad_none(el_photon_idxs,1))
# print(mu_photon_idxs.compute())
flag = ak.fill_none((mu_photon_idxs==el_photon_idxs), value=False)
print(ak.sum(flag).compute())
# print(ak.num(mu_photon_idxs, axis=0).compute())
# print(ak.sum(mu_photon_idxs, axis=0).compute())


0


In [88]:
idxs = [1,2,3]
for i in idxs:
    mu_photon_idxs = events.Muon.fsrPhotonIdxG[events.Muon.fsrPhotonIdxG != -1][:,(i-1):i]
    mu_photon_idxs = ak.flatten(ak.pad_none(mu_photon_idxs,1)) # get event length array of idxs with flatten
    for j in idxs:
        el_photon_idxs = events.Electron.photonIdxG[events.Electron.photonIdxG != -1][:,(j-1):j]
        el_photon_idxs = ak.flatten(ak.pad_none(el_photon_idxs,1)) # get event length array of idxs with flatten
        print(mu_photon_idxs.compute())
        print(el_photon_idxs.compute())
        flag = ak.fill_none((mu_photon_idxs==el_photon_idxs), value=False)
        # print(
        # print(ak.sum(flag).compute())

[0, None, None, None, None, None, None, ..., None, None, None, None, None, None]
[None, None, None, None, None, None, ..., None, None, None, None, None, None]
0
[0, None, None, None, None, None, None, ..., None, None, None, None, None, None]
[None, None, None, None, None, None, ..., None, None, None, None, None, None]
0
[0, None, None, None, None, None, None, ..., None, None, None, None, None, None]
[None, None, None, None, None, None, ..., None, None, None, None, None, None]
0
[None, None, None, None, None, None, ..., None, None, None, None, None, None]
[None, None, None, None, None, None, ..., None, None, None, None, None, None]
0


KeyboardInterrupt: 

In [92]:
idxs = [1,2,3]
# big = events.Electron.photonIdxG[events.Electron.photonIdxG != -1]
big = events.Electron.photonIdxG

print(big.compute())
for i in idxs:
    # el_photon_idxs = events.Electron.photonIdxG[events.Electron.photonIdxG != -1][:,(i-1):i]
    el_photon_idxs = events.Electron.photonIdxG[:,(i-1):i]

    print(el_photon_idxs.compute())

[[], [], [], [], [-1], [], [-1], [], [], ..., [], [-1], [], [], [], [], [], []]
[[], [], [], [], [-1], [], [-1], [], [], ..., [], [-1], [], [], [], [], [], []]
[[], [], [], [], [], [], [], [], [], ..., [], [], [], [], [], [], [], [], []]
[[], [], [], [], [], [], [], [], [], ..., [], [], [], [], [], [], [], [], []]


In [ ]:
idxs = [1,2,3]
big = events.Muon.fsrPhotonIdxG[events.Muon.fsrPhotonIdxG != -1]
print(big.compute())
for i in idxs:
    mu_photon_idxs = events.Muon.fsrPhotonIdxG[events.Muon.fsrPhotonIdxG != -1][:,(i-1):i]
    print(mu_photon_idxs.compute())

In [71]:
mu_photon_idxs = events.Muon.fsrPhotonIdxG[events.Muon.fsrPhotonIdxG != -1]
el_photon_idxs = events.Electron.photonIdxG[events.Electron.photonIdxG != -1]

print(ak.sum(ak.num(mu_photon_idxs, axis=1)>3).compute())
print(ak.sum(ak.num(mu_photon_idxs, axis=1)>1).compute())
print(ak.sum(ak.num(mu_photon_idxs, axis=1)>0).compute())
print(ak.sum(ak.num(el_photon_idxs, axis=1)>3).compute())
print(ak.sum(ak.num(el_photon_idxs, axis=1)>1).compute())
print(ak.sum(ak.num(el_photon_idxs, axis=1)>0).compute())

0
657
47502
0
1297
34940


In [77]:
events.Electron.photonIdxG?
events.Muon.fsrPhotonIdxG?

Type:            Array
String form:     dask.awkward<fsrPhotonIdxG, npartitions=1>
File:            /depot/cms/kernels/coffea2024/lib/python3.10/site-packages/dask_awkward/lib/core.py
Docstring:       Index of the associated FSR photon
Class docstring:
Partitioned, lazy, and parallel Awkward Array Dask collection.

The class constructor is not intended for users. Instead use
factory functions like :py:func:`~dask_awkward.from_parquet`,
:py:func:`~dask_awkward.from_json`, etc.

Within dask-awkward the ``new_array_object`` factory function is
used for creating new instances.

In [78]:
(events.Muon.fsrPhotonIdx >= 0).compute()

<Array [[True], [False, ...], ..., [False]] type='999000 * var * bool[param...'>

In [58]:
# print(mu_photon_idxs.compute())
# print(el_photon_idxs.compute())
# (mu_photon_idxs==el_photon_idxs).compute()
# print(ak.flatten(mu_photon_idxs).compute())
print(ak.flatten(ak.pad_none(mu_photon_idxs,1)).compute())


[0, None, None, None, None, None, None, ..., None, None, None, None, None, None]


In [67]:
ak.sum(events.Muon.fsrPhotonIdx!=events.Muon.fsrPhotonIdxG).compute()

48158

In [35]:
# mu_photon_idxs = ak.sum(mu_photon_idxs, axis=1)
# el_photon_idxs = ak.sum(el_photon_idxs, axis=1)
ak.sum(mu_photon_idxs==el_photon_idxs, axis=0).compute()

918383

In [ ]:
events.Muon.matched_fsrPhoton.delta_r(events.Electron.matched_photon).compute()

In [42]:
events.Muon.matched_fsrPhoton.fields

['dROverEt2', 'eta', 'phi', 'pt', 'relIso03', 'muonIdx', 'muonIdxG']

In [48]:
ak.sum(ak.fill_none((events.Muon.matched_fsrPhoton.relIso03 > 1.8),value=False)).compute()

3717